In [ ]:
pip install datasets

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [ ]:
# to store and read data from google drive
from google.colab import drive
from datasets import load_from_disk
import shutil
import os

In [ ]:
dataset = load_dataset("DarthReca/california_burned_areas", name="post-fire", trust_remote_code=True)
# dataset = load_dataset("DarthReca/california_burned_areas", name="pre-post-fire", trust_remote_code=True)

# Load dataset from Google Drive
# drive.mount('/content/drive')

# # Specify the directory where you want to store the dataset in Google Drive
# save_path = '/content/drive/My Drive/Colab Notebooks/california_burned_areas'

# # Make the directory if it doesn't exist
# os.makedirs(save_path, exist_ok=True)

# # Save the dataset to Google Drive
# dataset.save_to_disk(save_path)

# dataset = load_from_disk('/content/drive/My Drive/Colab Notebooks/california_burned_areas')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.68k [00:00<?, ?B/s]

california_burned_areas.py:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

512x512.hdf5:   0%|          | 0.00/5.88G [00:00<?, ?B/s]

chabud_test.h5:   0%|          | 0.00/667M [00:00<?, ?B/s]

...

Generating 1 split: 0 examples [00:00, ? examples/s]

Generating 2 split: 0 examples [00:00, ? examples/s]

Generating 3 split: 0 examples [00:00, ? examples/s]

Generating 4 split: 0 examples [00:00, ? examples/s]

Generating chabud split: 0 examples [00:00, ? examples/s]

In [ ]:
X_0 = dataset['0']['post_fire']
print('done')
X_1 = dataset['1']['post_fire']
print('done')
X_2 = dataset['2']['post_fire']
print('done')
X_3 = dataset['3']['post_fire']
print('done')
X_4 = dataset['4']['post_fire']
print('done')
X_c = dataset['chabud']['post_fire']

done
done
done
done
done


In [ ]:
m_0 = dataset['0']['mask']
print('done')
m_1 = dataset['1']['mask']
print('done')
m_2 = dataset['2']['mask']
print('done')
m_3 = dataset['3']['mask']
print('done')
m_4 = dataset['4']['mask']
print('done')
m_c = dataset['chabud']['mask']

done
done
done
done
done


In [ ]:
# from ChatGPT, add factor before split the data

# Functions to calculate the indices
# Small constant to avoid division by zero
epsilon = 1e-10

# Functions to calculate the indices with added epsilon
def ndvi(b4, b8):
    return (b8 - b4) / (b8 + b4 + epsilon)

def abai(b3, b11, b12):
    return (3 * b12 - 2 * b11 - 3 * b3) / (3 * b12 + 2 * b11 + 3 * b3 + epsilon)

def nbr(b2, b3, b8a, b12):
    return (b12 - b8a - b3 - b2) / (b12 + b8a + b3 + b2 + epsilon)

# Helper function to add indices to each image
def add_indices(image):
    # Convert to a NumPy array if it's not already
    image = np.array(image)

    # Extract the required bands
    b2, b3, b4, b8, b8a, b11, b12 = image[..., 1], image[..., 2], image[..., 3], image[..., 7], image[..., 8], image[..., 10], image[..., 11]
    ndvi_band = ndvi(b4, b8)
    abai_band = abai(b3, b11, b12)
    nbr_band = nbr(b2, b3, b8a, b12)

    # Stack indices as additional bands
    return np.dstack((image, ndvi_band, abai_band, nbr_band))

# Concatenate all images and masks
all_images = [X_0, X_1, X_2, X_3, X_4, X_c]
all_masks = [m_0, m_1, m_2, m_3, m_4, m_c]

# Add indices to each image
processed_images = []
for subset in all_images:
    for img in subset:
        processed_images.append(add_indices(img))

# Concatenate masks into a single list
processed_masks = [mask for subset in all_masks for mask in subset]

# Split into train and test sets
train_X, test_X, train_Y, test_Y = train_test_split(processed_images, processed_masks, test_size=0.3)

# Check the final shape of train_X
print(np.array(train_X).shape)  # Should be (421, 512, 512, 15)

(421, 512, 512, 15)


In [ ]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        # UNET repicated of https://arxiv.org/pdf/2211.12979 page 5.
        ## START OF ENCODING BLOCK ##
        self.orange = nn.Conv2d(in_channels=12, out_channels=64, kernel_size=7, padding='same') #-> change this to how many channels you will use
        self.red1 = nn.MaxPool2d(2)
        self.blue1 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding='same'),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(inplace=True),
                                   nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding='same')
        )
        self.red2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, padding=1)
        self.blue2 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding='same'),
                                   nn.BatchNorm2d(128),
                                   nn.ReLU(inplace=True),
                                   nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding='same')
        )
        self.red3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=2, padding=1)
        self.blue3 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding='same'),
                                   nn.BatchNorm2d(256),
                                   nn.ReLU(inplace=True),
                                   nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding='same')
        )
        self.red4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=2, padding=1)
        self.blue4 = nn.Sequential(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding='same'),
                                   nn.BatchNorm2d(512),
                                   nn.ReLU(inplace=True),
                                   nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding='same')
        )
        ## END OF ENCODER BLOCK

        ## START OF DECODER BLOCK ##

        self.green1 = nn.ConvTranspose2d(in_channels=512, out_channels=512, kernel_size=2, stride=2)
        self.upblue1 = nn.Sequential(nn.Conv2d(in_channels=768, out_channels=256, kernel_size=3, padding='same'),
                                   nn.BatchNorm2d(256),
                                   nn.ReLU(inplace=True),
                                   nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding='same'))
        self.green2 = nn.ConvTranspose2d(in_channels=256, out_channels=256, kernel_size=2, stride=2)
        self.upblue2 = nn.Sequential(nn.Conv2d(in_channels=384, out_channels=128, kernel_size=3, padding='same'),
                                   nn.BatchNorm2d(128),
                                   nn.ReLU(inplace=True),
                                   nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding='same'))
        self.green3 = nn.ConvTranspose2d(in_channels=128, out_channels=128, kernel_size=2, stride=2)
        self.upblue3 = nn.Sequential(nn.Conv2d(in_channels=192, out_channels=64, kernel_size=3, padding='same'),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(inplace=True),
                                   nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding='same'))
        self.green4 = nn.ConvTranspose2d(in_channels=64, out_channels=64, kernel_size=2, stride=2)
        self.upblue4 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=32, kernel_size=3, padding='same'),
                                   nn.BatchNorm2d(32),
                                   nn.ReLU(inplace=True),
                                   nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding='same'))
        self.upblue5 = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding='same'),
                                   nn.BatchNorm2d(32),
                                   nn.ReLU(inplace=True),
                                   nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding='same'))
        self.upblue6 = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding='same'),
                                   nn.BatchNorm2d(32),
                                   nn.ReLU(inplace=True),
                                   nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, padding='same'))

        ## END OF DECODER BLOCK
        self.final_conv = nn.Conv2d(16, 1, kernel_size=1)
    def forward(self, x):
        #shape of x -> 12, 512, 512
        orange_op = self.orange(x)
        #print('orange_op : ',orange_op.shape) -> torch.Size([1, 64, 512, 512])
        red1_op = self.red1(orange_op)
        # print('red1_op : ',red1_op.shape) -> torch.Size([1, 64, 256, 256])
        blue1_op = self.blue1(self.blue1(red1_op)) + red1_op
        # print('blue1_op : ',blue1_op.shape) -> torch.Size([1, 64, 256, 256])
        red2_op = self.red2(blue1_op)
        # print('red2_op : ',red2_op.shape) -> torch.Size([1, 128, 128, 128])
        blue2_op = self.blue2(self.blue2(self.blue2(red2_op))) + red2_op
        # print('blue2_op : ',blue2_op.shape) -> torch.Size([1, 128, 128, 128])
        red3_op = self.red3(blue2_op)
        # print('red3_op : ',red3_op.shape) -> torch.Size([1, 256, 64, 64])
        blue3_op = self.blue3(self.blue3(self.blue3(self.blue3(self.blue3(red3_op))))) + red3_op
        # print('blue3_op : ',blue3_op.shape) -> torch.Size([1, 256, 64, 64])
        red4_op = self.red4(blue3_op)
        # print('red4_op : ',red4_op.shape) -> torch.Size([1, 512, 32, 32])
        blue4_op = self.blue4(self.blue4(red4_op)) + red4_op
        # print('blue4_op : ',blue4_op.shape) -> torch.Size([1, 512, 32, 32])
        up1_op = self.upblue1(torch.cat((self.green1(blue4_op), blue3_op), dim=1))
        # print('up1_op : ',up1_op.shape) -> torch.Size([1, 256, 64, 64])
        up2_op = self.upblue2(torch.cat((self.green2(up1_op), blue2_op), dim=1))
        # print('up2_op : ',up2_op.shape) -> torch.Size([1, 128, 128, 128])
        up3_op = self.upblue3(torch.cat((self.green3(up2_op), blue1_op), dim=1))
        # print('up3_op : ',up3_op.shape) -> torch.Size([1, 64, 256, 256])
        up4_op = self.upblue6(self.upblue5(self.upblue4(torch.cat((self.green4(up3_op), orange_op), dim=1))))
        # print('up4_op : ',up4_op.shape) -> torch.Size([1, 16, 512, 512])
        return torch.sigmoid(self.final_conv(up4_op))


model = UNet()

In [ ]:
from torch.utils.data import DataLoader, Dataset

class ImageData(Dataset):
    def __init__(self, images, masks):
        self.images = images
        self.masks = masks

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        mask = self.masks[idx]
        tensor_image = torch.tensor(image[:, :, [1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14]]).float().permute(2, 0, 1) #-> make changes to what channels you want to include
        tensor_mask = torch.tensor(mask).float().permute(2, 0, 1)
        return tensor_image, tensor_mask

train_dataset = ImageData(images=train_X, masks=train_Y)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

test_dataset = ImageData(images=test_X, masks=test_Y)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)


In [ ]:
def precision_score_(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    total_pixel_pred = np.sum(pred_mask)
    precision = np.mean(intersect/total_pixel_pred)
    return round(precision, 3)

def recall_score_(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    total_pixel_truth = np.sum(groundtruth_mask)
    recall = np.mean(intersect/total_pixel_truth)
    return round(recall, 3)

def dice_loss(groundtruth_mask, pred_mask):
    intersect = torch.sum(pred_mask * groundtruth_mask)
    total_sum = torch.sum(pred_mask) + torch.sum(groundtruth_mask)
    dice = 1 - (2 * intersect / (total_sum + 1e-6))  # Avoid division by zero
    return dice

def iou_(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    union = np.sum(pred_mask) + np.sum(groundtruth_mask) - intersect
    return round(np.mean(intersect/union), 3)

In [ ]:
import torch.optim as optim
criterion = nn.BCELoss()  # Use BCEWithLogitsLoss for binary segmentation
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training Loop
num_epochs = 10
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    all_preds = []
    all_masks = []

    for images, masks in tqdm(train_loader):
        images, masks = images.to(device), masks.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, masks)  # Cast masks to long if needed
        dice = dice_loss(masks, outputs)
        total_loss = loss + dice
        epoch_loss += total_loss.item()


        # Backpropagation and optimization
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        preds = (outputs > 0.5).float()
        all_preds.extend(preds.squeeze(1).cpu().numpy())
        all_masks.extend(masks.squeeze(1).cpu().numpy())

    recall = recall_score_(np.array(all_preds), np.array(all_masks))
    precision = precision_score_(np.array(all_preds), np.array(all_masks))
    iou = iou_(np.array(all_preds), np.array(all_masks))

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader)}")
    print(f"Precision: {recall:.4f}, Recall: {precision:.4f}")
    if recall != 0. and precision != 0.:
        print(f'F1 - score : {2/((1/recall) + (1/precision))}')
    print(f"IOU : {iou}")


100%|██████████| 53/53 [09:32<00:00, 10.79s/it]


Epoch [1/10], Loss: 1.246228738775793
Precision: 0.5730, Recall: 0.3400
F1 - score : 0.4267689004095519


100%|██████████| 53/53 [09:21<00:00, 10.60s/it]


Epoch [2/10], Loss: 0.9898301362991333
Precision: 0.4950, Recall: 0.5730
F1 - score : 0.531151694079501


100%|██████████| 53/53 [09:28<00:00, 10.73s/it]


...

Epoch [8/10], Loss: 0.73615488972304
Precision: 0.6360, Recall: 0.5930
F1 - score : 0.6137477491638619


100%|██████████| 53/53 [09:36<00:00, 10.88s/it]


Epoch [9/10], Loss: 0.659788857653456
Precision: 0.6810, Recall: 0.6210
F1 - score : 0.6496175044820451


100%|██████████| 53/53 [09:42<00:00, 10.99s/it]


Epoch [10/10], Loss: 0.6603563298594277
Precision: 0.6750, Recall: 0.6200
F1 - score : 0.646332054387972


In [ ]:
model.eval()
epoch_loss = 0
all_preds = []
all_masks = []

for images, masks in tqdm(test_loader):
    images, masks = images.to(device), masks.to(device)

    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, masks)  # Cast masks to long if needed
    dice = dice_loss(masks, outputs)
    total_loss = loss + dice
    epoch_loss += total_loss.item()


    # Backpropagation and optimization
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    preds = (outputs > 0.5).float()
    all_preds.extend(preds.squeeze(1).cpu().numpy())
    all_masks.extend(masks.squeeze(1).cpu().numpy())

recall = recall_score_(np.array(all_preds), np.array(all_masks))
precision = precision_score_(np.array(all_preds), np.array(all_masks))
model.eval()
epoch_loss = 0
all_preds = []
all_masks = []

for prefires, postfires, masks in tqdm(test_loader):
    prefires, postfires, masks = prefires.to(device), postfires.to(device), masks.to(device)

    # Forward pass
    outputs = model(prefires, postfires)
    # loss = criterion(outputs, masks)  # Cast masks to long if needed
    # dice = dice_loss(masks, outputs)
    # total_loss = loss + dice
    # epoch_loss += total_loss.item()


    # # Backpropagation and optimization
    # optimizer.zero_grad()
    # total_loss.backward()
    # optimizer.step()
    preds = (outputs > 0.5).float()
    all_preds.extend(preds.squeeze(1).cpu().numpy())
    all_masks.extend(masks.squeeze(1).cpu().numpy())

recall = recall_score_(np.array(all_preds), np.array(all_masks))
precision = precision_score_(np.array(all_preds), np.array(all_masks))
iou = iou_(np.array(all_preds), np.array(all_masks))

print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader)}")
print(f"Precision: {recall:.4f}, Recall: {precision:.4f}")
if recall != 0. and precision != 0.:
    print(f'F1 - score : {2/((1/recall) + (1/precision))}')
print(f"IOU : {iou}")

100%|██████████| 23/23 [04:06<00:00, 10.71s/it]


Epoch [10/10], Loss: 0.31242832597696557
Precision: 0.7440, Recall: 0.5660
F1 - score : 0.6429068669724881


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(np.array(all_preds).flatten(), np.array(all_masks).flatten()))

              precision    recall  f1-score   support

         0.0       0.98      0.95      0.96  43372819
         1.0       0.57      0.74      0.64   4075245

    accuracy                           0.93  47448064
   macro avg       0.77      0.85      0.80  47448064
weighted avg       0.94      0.93      0.93  47448064

